# protodash

I could not get the aix360 version here to work because it has dependencies of tf 1.x 

I realized protodash does not run any tf objects so I decided to just make my own notebook containing all the stuff needed to make it work copying code from teh aix360 repo.

https://github.com/Trusted-AI/AIX360

In [1]:
# from https://github.com/Trusted-AI/AIX360/blob/master/aix360/algorithms/die.py
import abc
import sys

# Ensure compatibility with Python 2/3
if sys.version_info >= (3, 4):
    ABC = abc.ABC
else:
    ABC = abc.ABCMeta(str('ABC'), (), {})


class DIExplainer(ABC):
    """
    DIExplainer is the base class for Directly Interpretable unsupervised explainers (DIE).
    Such explainers generally rely on unsupervised techniques to explain datasets and model predictions.
    Examples include DIP-VAE[#1]_, Protodash[#2]_, etc.
    References:
        .. [#1] Variational Inference of Disentangled Latent Concepts from Unlabeled Observations (DIP-VAE), ICLR 2018.
         Kumar, Sattigeri, Balakrishnan. https://arxiv.org/abs/1711.00848
        .. [#2] ProtoDash: Fast Interpretable Prototype Selection, 2017.
        Karthik S. Gurumoorthy, Amit Dhurandhar, Guillermo Cecchi.
        https://arxiv.org/abs/1707.01212
    """

    def __init__(self, *argv, **kwargs):
        """
        Initialize a DIExplainer object.
        ToDo: check common steps that need to be distilled here.
        """

    @abc.abstractmethod
    def set_params(self, *argv, **kwargs):
        """
        Set parameters for the explainer.
        """
        raise NotImplementedError

    @abc.abstractmethod
    def explain(self, *argv, **kwargs):
        """
        Explain the data or model.
        """
        raise NotImplementedError

In [2]:
#from: https://github.com/Trusted-AI/AIX360/blob/master/aix360/algorithms/protodash/PDASH_utils.py
import numpy as np
from numpy import array, ndarray
import xport
from sklearn.preprocessing import OneHotEncoder
from scipy.spatial.distance import cdist
import cvxpy as cp
from scipy import sparse
from osqp import OSQP


# Removing dependence on cxvopt and qpsolver packages. 
# from cvxopt.solvers import qp
# from cvxopt import matrix, spmatrix
# from qpsolvers import solve_qp
# def runOptimiser(K, u, preOptw, initialValue, maxWeight=10000):
#     """
#     Args:
#         K (double 2d array): Similarity/distance matrix
#         u (double array): Mean similarity of each prototype
#         preOptw (double): Weight vector
#         initialValue (double): Initialize run
#         maxWeight (double): Upper bound on weight

#     Returns:
#         Prototypes, weights and objective values
#     """
#     d = u.shape[0]
#     lb = np.zeros((d, 1))
#     ub = maxWeight * np.ones((d, 1))
#     x0 = np.append( preOptw, initialValue/K[d-1, d-1] )

#     G = np.vstack((np.identity(d), -1*np.identity(d)))
#     h = np.vstack((ub, -1*lb))

#     #     Solve a QP defined as follows:
#     #         minimize
#     #             (1/2) * x.T * P * x + q.T * x
#     #         subject to
#     #             G * x <= h
#     #             A * x == b
#     sol = solve_qp(K, -u, G, h, A=None, b=None, solver='cvxopt', initvals=x0)

#     # compute objective function value
#     x = sol.reshape(sol.shape[0], 1)
#     P = K
#     q = - u.reshape(u.shape[0], 1)
#     obj_value = 1/2 * np.matmul(np.matmul(x.T, P), x) + np.matmul(q.T, x)
#     return(sol, obj_value[0,0])


def runOptimiser(K, u, preOptw, initialValue, optimizer, maxWeight=10000):
    """
    Args:
        K (double 2d array): Similarity/distance matrix
        u (double array): Mean similarity of each prototype
        preOptw (double): Weight vector
        initialValue (double): Initialize run
        optimizer (string): qpsolver ('cvxpy' or 'osqp')
        maxWeight (double): Upper bound on weight
        
    Returns:
        Prototypes, weights and objective values
    """
    
    #     Standard QP:
    #         minimize
    #             (1/2) * x.T * P * x + q.T * x
    #         subject to
    #             G * x <= h
    #             A * x == b
    
    #     QP Solved by Protodash:
    #         minimize
    #             (1/2) * x.T * K * x + (-u).T * x
    #         subject to
    #             G * x <= h
    
    assert (optimizer=='cvxpy' or optimizer=='osqp'), "Please set optimizer as 'cvxpy' or 'osqp'"
    
    d = u.shape[0]
    lb = np.zeros((d, 1))
    ub = maxWeight * np.ones((d, 1))
    
    # x0 = initial value, provided optimizer supports it. 
    x0 = np.append( preOptw, initialValue/K[d-1, d-1] )
    
    G = np.vstack((np.identity(d), -1*np.identity(d)))
    h = np.vstack((ub, -1*lb)).ravel()

    # variable shapes: K = (d,d), u = (d,) G = (2d, d), h = (2d,)
        
    if (optimizer == 'cvxpy'):
        x = cp.Variable(d)
        prob = cp.Problem(cp.Minimize((1/2)*cp.quad_form(x, K) + (-u).T@x), [G@x <= h])
        prob.solve()
        
        xv = x.value.reshape(-1, 1)
        xreturn = x.value
        
    elif (optimizer == 'osqp'): 
        
        Ks = sparse.csc_matrix(K)
        Gs = sparse.csc_matrix(G)
        l_inf = -np.inf * np.ones(len(h))
        
        solver = OSQP() 
        solver.setup(P=Ks, q=-u, A=Gs, l=l_inf, u=h, eps_abs=1e-4, eps_rel=1e-4, polish= True, verbose=False) 
        solver.warm_start(x=x0) 
        res = solver.solve() 
        
        xv = res.x.reshape(-1, 1) 
        xreturn = res.x 
        
    # compute objective function value        
    P = K
    q = - u.reshape(-1, 1)
    obj_value = 1/2 * np.matmul(np.matmul(xv.T, P), xv) + np.matmul(q.T, xv)
    
    return(xreturn, obj_value[0,0])



def get_Processed_NHANES_Data(filename):
    """
    Args:
        filename (str): Enter NHANES filename
    Returns:
        One hot encoded features and original input
    """
    # returns original and one hot encoded data
    # Input: XPT filename e.g. 2_H.XPT)
    # output:
    # One hot endcoded, e.g. (5924 x 145)
    # original, e.g. (5924 x 9)

    with open(filename, 'rb') as f:
        original = xport.to_numpy(f)

    # replace nan's with 0's.
    original[np.isnan(original)] = 0

    # delete 1st column (contains sequence numbers)
    original = original[:, 1:]

    # one hot encoding of all columns/features
    onehot_encoder = OneHotEncoder(sparse=False)
    onehot_encoded = onehot_encoder.fit_transform(original)

    # return one hot encoded and original data
    return (onehot_encoded, original)


def get_Gaussian_Data(nfeat, numX, numY):
    """
    Args:
        nfeat (int): Number of features
        numX (int): Size of X
        numY (int): Size of Y
    Returns:
        Datasets X and Y
    """
    np.random.seed(0)
    X = np.random.normal(0.0, 1.0, (numX, nfeat))
    Y = np.random.normal(0.0, 1.0, (numY, nfeat))

    for i in range(numX):
        X[i, :] = X[i, :] / np.linalg.norm(X[i, :])

    for i in range(numY):
        Y[i, :] = Y[i, :] / np.linalg.norm(Y[i, :])

    return(X, Y)


# expects X & Y in (observations x features) format

def HeuristicSetSelection(X, Y, m, kernelType, sigma, optimizer):
    """
    Main prototype selection function.
    Args:
        X (double 2d array): Dataset to select prototypes from
        Y (double 2d array): Dataset to explain
        m (double): Number of prototypes
        kernelType (str): Gaussian, linear or other
        sigma (double): Gaussian kernel width
        optimizer (string): qpsolver ('cvxpy' or 'osqp')
    Returns:
        Current optimum, the prototypes and objective values throughout selection
    """
    numY = Y.shape[0]
    numX = X.shape[0]
    allY = np.array(range(numY))

    # Store the mean inner products with X
    if kernelType == 'Gaussian':
        meanInnerProductX = np.zeros((numY, 1))
        for i in range(numY):
            Y1 = Y[i, :]
            Y1 = Y1.reshape(Y1.shape[0], 1).T
            distX = cdist(X, Y1)
            meanInnerProductX[i] = np.sum( np.exp(np.square(distX)/(-2.0 * sigma**2)) ) / numX
    else:
        M = np.dot(Y, np.transpose(X))
        meanInnerProductX = np.sum(M, axis=1) / M.shape[1]

    # move to features x observation format to be consistent with the earlier code version
    X = X.T
    Y = Y.T

    # Intialization
    S = np.zeros(m, dtype=int)
    setValues = np.zeros(m)
    sizeS = 0
    currSetValue = 0.0
    currOptw = np.array([])
    currK = np.array([])
    curru = np.array([])
    runningInnerProduct = np.zeros((m, numY))

    while sizeS < m:

        remainingElements = np.setdiff1d(allY, S[0:sizeS])

        newCurrSetValue = currSetValue
        maxGradient = 0

        for count in range(remainingElements.shape[0]):

            i = remainingElements[count]
            newZ = Y[:, i]

            if sizeS == 0:

                if kernelType == 'Gaussian':
                    K = 1
                else:
                    K = np.dot(newZ, newZ)

                u = meanInnerProductX[i]
                w = np.max(u / K, 0)
                incrementSetValue = -0.5 * K * (w ** 2) + (u * w)

                if (incrementSetValue > newCurrSetValue) or (count == 1):
                    # Bookeeping
                    newCurrSetValue = incrementSetValue
                    desiredElement = i
                    newCurrOptw = w
                    currK = K

            else:
                recentlyAdded = Y[:, S[sizeS - 1]]

                if kernelType == 'Gaussian':
                    distnewZ = np.linalg.norm(recentlyAdded-newZ)
                    runningInnerProduct[sizeS - 1, i] = np.exp( np.square(distnewZ)/(-2.0 * sigma**2 ) )
                else:
                    runningInnerProduct[sizeS - 1, i] = np.dot(recentlyAdded, newZ)

                innerProduct = runningInnerProduct[0:sizeS, i]
                if innerProduct.shape[0] > 1:
                    innerProduct = innerProduct.reshape((innerProduct.shape[0], 1))

                gradientVal = meanInnerProductX[i] - np.dot(currOptw, innerProduct)

                if (gradientVal > maxGradient) or (count == 1):
                    maxGradient = gradientVal
                    desiredElement = i
                    newinnerProduct = innerProduct[:]

        S[sizeS] = desiredElement

        curru = np.append(curru, meanInnerProductX[desiredElement])

        if sizeS > 0:

            if kernelType == 'Gaussian':
                selfNorm = array([1.0])
            else:
                addedZ = Y[:, desiredElement]
                selfNorm = array( [np.dot(addedZ, addedZ)] )

            K1 = np.hstack((currK, newinnerProduct))

            if newinnerProduct.shape[0] > 1:
                selfNorm = selfNorm.reshape((1,1))
            K2 = np.vstack( (K1, np.hstack((newinnerProduct.T, selfNorm))) )

            currK = K2
            if maxGradient <= 0:
                #newCurrOptw = np.vstack((currOptw[:], np.array([0])))
                newCurrOptw = np.append(currOptw, [0], axis=0)
                newCurrSetValue = currSetValue
            else:
                [newCurrOptw, value] = runOptimiser(currK, curru, currOptw, maxGradient, optimizer)
                newCurrSetValue = -value

        currOptw = newCurrOptw
        if type(currOptw) != np.ndarray:
            currOptw = np.array([currOptw])

        currSetValue = newCurrSetValue

        setValues[sizeS] = currSetValue
        sizeS = sizeS + 1

    return(currOptw, S, setValues)

In [3]:
#from https://github.com/Trusted-AI/AIX360/blob/master/aix360/algorithms/protodash/PDASH.py
class ProtodashExplainer(DIExplainer):
    """
    ProtodashExplainer provides exemplar-based explanations for summarizing datasets as well
    as explaining predictions made by an AI model. It employs a fast gradient based algorithm
    to find prototypes along with their (non-negative) importance weights. The algorithm minimizes the maximum
    mean discrepancy metric and has constant factor approximation guarantees for this weakly submodular function. [#]_.
    References:
        .. [#] `Karthik S. Gurumoorthy, Amit Dhurandhar, Guillermo Cecchi,
           "ProtoDash: Fast Interpretable Prototype Selection"
           <https://arxiv.org/abs/1707.01212>`_
    """

    def __init__(self):
        """
        Constructor method, initializes the explainer
        """
        super(ProtodashExplainer, self).__init__()

    def set_params(self, *argv, **kwargs):
        """
        Set parameters for the explainer.
        """
        pass

    def explain(self, X, Y, m, kernelType='other', sigma=2, optimizer='cvxpy'):
        """
        Return prototypes for data X, Y.
        Args:
            X (double 2d array): Dataset you want to explain.
            Y (double 2d array): Dataset to select prototypical explanations from.
            m (int): Number of prototypes
            kernelType (str): Type of kernel (viz. 'Gaussian', / 'other')
            sigma (double): width of kernel
            optimizer (string): qpsolver ('cvxpy' or 'osqp')
            
        Returns:
            m selected prototypes from X and their (unnormalized) importance weights
        """
        return( HeuristicSetSelection(X, Y, m, kernelType, sigma, optimizer) )

In [4]:
explainer = ProtodashExplainer()

In [5]:
#load some data to check
import pandas as pd

diabetes_test = pd.read_csv('diabetes.csv')

diabetes_test

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
...,...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63,0
764,2,122,70,27,0,36.8,0.340,27,0
765,5,121,72,23,112,26.2,0.245,30,0
766,1,126,60,0,0,30.1,0.349,47,1


In [27]:
#np_diabetes_test = diabetes_test.drop(columns=["Outcome"], axis = 1).to_numpy()
np_diabetes_test = diabetes_test.to_numpy()

In [29]:
# replace nan's (missing values) with 0's
original = np_diabetes_test
original[np.isnan(original)] = 0

In [32]:
(W, S, _) = explainer.explain(original, original, m=20, kernelType='Gaussian', sigma=2, optimizer='osqp')

In [34]:
diabetes_prototypes = diabetes_test.iloc[S,:].copy()

diabetes_prototypes["Weights of Prototypes"] = np.around(W/np.sum(W), 2) 

In [35]:
diabetes_prototypes

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome,Weights of Prototypes
643,4,90,0,0,0,28.0,0.610,31,0,0.07
752,3,108,62,24,0,26.0,0.223,25,0,0.06
430,2,99,0,0,0,22.2,0.108,23,0,0.06
262,4,95,70,32,0,32.1,0.612,24,0,0.06
739,1,102,74,0,0,39.5,0.293,42,1,0.05
278,5,114,74,0,0,24.9,0.744,57,0,0.05
451,2,134,70,0,0,28.9,0.542,23,1,0.05
529,0,111,65,0,0,24.6,0.660,31,0,0.05
179,5,130,82,0,0,39.1,0.956,37,1,0.05
494,3,80,0,0,0,0.0,0.174,22,0,0.05
